In [ ]:
# load data
import nltk
nltk.download()
import time
from nltk.corpus import stopwords
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize
#from nltk.stem import WordNetLemmatizer
#wordnet=WordNetLemmatizer()
from nltk.stem.porter import PorterStemmer
porter=PorterStemmer()
import string
def preprocess(f,req=0):
    filename = f
    file = open(filename, 'rt')
    text = file.read()
    file.close()
    # split into sentences
    sentences = sent_tokenize(text)
    #print(sentences[0])
    #split into words
    w=word_tokenize(text)
    #print(w[:10])
    #removing puctuations and stopwords

    stop=stopwords.words("english")
    words=[]
    for i in w:
        if (i not in string.punctuation) and (i not in stop):
            #k=wordnet.lemmatize(i)
            k=porter.stem(i)
            #print(k)
            #if k!=i:
               # print(k,"-",i)
            words.append(k)
    #print(words[:10])
    #print("before",sentences[0])

    for i in range(len(sentences)):
        sentences[i]=sentences[i].lower()
        for j in sentences[i]:
            if j in string.punctuation:
                sentences[i]=sentences[i].replace(j," ")
                
        print(sentences[i])
    #print(sentences[0])
    if req==0:
        return(sentences)
    else:
        k=(len(min(words,key=len))+len(max(words,key=len)))//2
        str=""
        for i in words:
            str=str+i
        #print(str)
        for i in string.punctuation:
            str=str.replace(i,"")
        return(str,k)
#preprocess("data1.txt",0)


        


In [ ]:
class RollingHash:
    def __init__(self, text, sizeWord):
        self.text = text
        self.hash = 0 
        self.sizeWord = sizeWord
        self.hashtable=[]

        for i in range(0, sizeWord):
            #ord maps the character to a number
            #subtract out the ASCII value of "a" to start the indexing at zero
            self.hash += (ord(self.text[i]) - ord("a")+1)*(33**(sizeWord - i -1))

        #start index of current window
        self.window_start = 0
        #end of index window
        self.window_end = sizeWord

    def move_window(self):
        if self.window_end <= len(self.text) - 1:
            #remove left letter from hash value
            self.hash -= (ord(self.text[self.window_start]) - ord("a")+1)*33**(self.sizeWord-1)
            self.hash *= 33
            self.hash += ord(self.text[self.window_end])- ord("a")+1
            self.window_start += 1
            self.window_end += 1

    def window_text(self):
        return self.text[self.window_start:self.window_end]

def rabin_karp(word, text):
    if word == "" or text == "":
        return None
    if len(word) > len(text):
        return None

    rolling_hash = RollingHash(text, len(word))
    #print(rolling_hash)
    word_hash = RollingHash(word, len(word))
    #print(word_hash)
    #word_hash.move_window()

    for i in range(len(text) - len(word) + 1):
        if rolling_hash.hash == word_hash.hash:
            if rolling_hash.window_text() == word:
                return i
        rolling_hash.move_window()
    return None
print(rabin_karp('hash','rolling-hash'))
def rabin_karp2(text,k=5):
    rolling_hash = RollingHash(text,k)
    for i in range(len(text) - k + 1):
        rolling_hash.hashtable.append(rolling_hash.hash)
        rolling_hash.move_window()
    return(rolling_hash.hashtable)
rabin_karp2("aabab",3)
rabin_karp2("aaaab",3)
        

In [ ]:
def kgram(file1,file2):
    f1=preprocess(file1,1)
    #print("f1:",f1)
    f2=preprocess(file2,1)
    k=f1[1]
    f1=f1[0]
    f2=f2[0]
    print("k=",k)
    
    #f1="plagiarisminstanceusingcloselyimitatinglanguagethoughtsanotherauthorwithoutauthorization"
    #f2="plagiarismcopyingideaswordsanotherpersonwithoutgivingcreditthatperson"
    #print("f2:",f2)
    table1=rabin_karp2(f1,10)
    a=(len(table1))
    table2=rabin_karp2(f2,10)
    b=(len(table2))
    if a<b:
        n=table1
        m=table2
    else:
        n=table2
        m=table1
    val=0
    for i in n:
        val=val+m.count(i)
    #print(val)
    return ((2*val)/(a+b))*100
    
#print(kgram())

In [ ]:
def main():
    file1=input("enter filename 1:")
    file2=input("enter filename 2:")
    sentences=preprocess(file1,0)
    f=open(file2,"r+")
    content=f.read()
    content=content.lower()
    for i in content:
        if i in string.punctuation:
            content=content.replace(i," ")
    #print(content)
    

    amount=0
    start=time.time()
    for i in sentences:
        temp=rabin_karp(i,content)
        if temp!=None:
            amount=amount+len(i)
    
    print("percentage of plagiarism using senteces=",(amount/len(content))*100)
    print("runtime=",time.time()-start)
    start=time.time()
    print("percentage of plagiarism using k-grams(words)=",kgram(file1,file2))
    print("runtime=",time.time()-start)
    
main()